<a href="https://colab.research.google.com/github/manglesh001/DL_Assignment_2/blob/main/DL_Ass_2_PartA_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

manglesh001_inaturalist12k_path = kagglehub.dataset_download('manglesh001/inaturalist12k')

print('Data source import complete.')


In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import wandb
from sklearn.model_selection import train_test_split


In [ ]:
# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

In [ ]:
!wandb login 6001619563748a57b4114b0bb090fd4129ba6122

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


**Q1-** Build a small CNN model consisting of 5 convolution layers. Each convolution layer would be followed by an activation and a max-pooling layer.

After 5 such conv-activation-maxpool blocks, you should have one dense layer followed by the output layer containing 10 neurons (1 for each of the 10 classes).

In [ ]:

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes=10, filter_counts=[32, 64, 128, 256, 512], filter_sizes=[(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)],
                 activation_func=nn.ReLU(), fc_neurons=512, use_batch_norm=True,dropout_rate=0.2):
        super(CNNModel, self).__init__()
        in_channels = 3  #RGB channels
        # Store layers in List
        self.features = nn.ModuleList()
        # doing 5 conv-act-maxpool blocks
        for i in range(5):
            # Add  2D convolution layer
            conv = nn.Conv2d(in_channels, filter_counts[i], kernel_size=filter_sizes[i], padding='same')
            self.features.append(conv)
            # Add batch normalization
            if use_batch_norm:
                self.features.append(nn.BatchNorm2d(filter_counts[i]))
            # Add activation function
            self.features.append(activation_func)
            # Add max pooling layer
            self.features.append(nn.MaxPool2d(kernel_size=2, stride=2))
            # Update in_channels for the next layer
            in_channels = filter_counts[i]
        # Adaptive pooling to handle different input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(filter_counts[-1], fc_neurons),activation_func,
            nn.Dropout(dropout_rate) if dropout_rate > 0 else nn.Identity(),
            nn.Linear(fc_neurons, num_classes)
        )

    def forward(self, x):
        # Pass input through feature layers
        for layer in self.features:
            x = layer(x)
        # Global average pooling
        x = self.adaptive_pool(x)
        # Flatten the tensor
        x = torch.flatten(x, 1)
        # Pass through classifier
        x = self.classifier(x)
        return x